In [1]:
import add_packages
import config
from pprint import pprint
import os

from my_langchain import (
  document_loaders, text_splitters, text_embedding_models, vectorstores, 
  chat_models, prompts, utils, output_parsers, agents
)

# Usecase

## Q&A with RAG 

### Base

In [2]:
import bs4
from langchain import hub
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_core.messages import AIMessage, HumanMessage

In [3]:
# Indexing: Load
bs4_strainer = bs4.SoupStrainer(class_=("post-title", "post-header", "post-content"))
loader = document_loaders.web_base_loader(
  web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
  bs_kwargs={"parse_only": bs4_strainer}
)
docs = loader.load()

# Indexing: Split
text_splitter = text_splitters.recursive_character_text_splitter(
  chunk_size=1000, chunk_overlap=200, add_start_index=True,
)
all_splits = text_splitter.split_documents(docs)

In [2]:
qdrant_instance = vectorstores.QdrantWrapper(
  collection_name="my-rag",
  qdrant_host=os.getenv("QDRANT_HOST"),
  qdrant_api_key=os.getenv("QDRANT_API_KEY"),
  default_search_type="similarity",
  default_search_kwargs={"k": 6},
  retriever_tool_name="search_state_of_union",
  retriever_tool_description="Searches and returns excerpts from the 2022 State of the Union.",
)

2024-03-07 10:45:10.146 | INFO     | my_langchain.vectorstores:__init__:99 - Found collection: `my-rag`.


In [6]:
# Indexing: Store
qdrant_instance.vector_store.add_documents(documents=all_splits)

In [7]:
# Retrieval and Generation: Retrieve
query = "What are the approaches to Task Decomposition?"
retrieved_docs = qdrant_instance.invoke_retriever(query)

In [8]:
# Retrieval and Generation: Generate

chat = chat_models.chat_openai

prompt = prompts.rag_prompt

rag_chain = (
  {
    "context": qdrant_instance.retriever | utils.format_docs, 
    "question": RunnablePassthrough()
  }
  | prompt
  | chat
  | output_parsers.str_output_parser()
)

example_messages = prompt.invoke(
  {
    "context": "filter context",
    "question": "filter question"
  }
).to_messages()

In [9]:
for chunk in rag_chain.stream("What is Task Decomposition?"):
  print(chunk, end="", flush=True)

Task decomposition involves breaking down complex tasks into smaller and simpler steps to enhance model performance. Techniques like Chain of Thought and Tree of Thoughts help in transforming big tasks into manageable ones by exploring multiple reasoning possibilities at each step. Task decomposition can be done using simple prompting, task-specific instructions, or human inputs.

### Add Source

In [10]:
# Adding sources
rag_chain_from_dos = (
  RunnablePassthrough.assign(context=(lambda x: utils.format_docs(x["context"])))
  | prompt
  | chat
  | output_parsers.str_output_parser()
)
rag_chain_with_source = RunnableParallel(
  {
    "context": qdrant_instance.retriever,
    "question": RunnablePassthrough()
  }
).assign(answer=rag_chain_from_dos)

In [17]:
rag_chain_with_source.invoke("What is Task Decomposition?")

{'context': [Document(page_content='Fig. 1. Overview of a LLM-powered autonomous agent system.\nComponent One: Planning#\nA complicated task usually involves many steps. An agent needs to know what they are and plan ahead.\nTask Decomposition#\nChain of thought (CoT; Wei et al. 2022) has become a standard prompting technique for enhancing model performance on complex tasks. The model is instructed to “think step by step” to utilize more test-time computation to decompose hard tasks into smaller and simpler steps. CoT transforms big tasks into multiple manageable tasks and shed lights into an interpretation of the model’s thinking process.', metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/', 'start_index': 1585, '_id': '3e36d9d5-a5f2-48cd-9bf9-0e1d127bf068', '_collection_name': 'my-rag'}),
  Document(page_content='Tree of Thoughts (Yao et al. 2023) extends CoT by exploring multiple reasoning possibilities at each step. It first decomposes the problem into multip

### Add chat history

In [11]:
contextualize_q_chain = (
  prompts.contextualize_q_prompt 
  | chat
  | output_parsers.str_output_parser()
)

def contextualized_question(input: dict):
  if input.get("chat_history"):
    return contextualize_q_chain
  else:
    return input["question"]

rag_chain = (
  RunnablePassthrough.assign(
    context=contextualized_question | qdrant_instance.retriever | utils.format_docs
  )
  | prompts.qa_prompt
  | chat
)


In [12]:
chat_history = []

questions = [
  "What is Task Decomposition?",
  "What are common ways of doing it?"
]

for question in questions:
  ai_msg = rag_chain.invoke({
    "question": question, "chat_history": chat_history
  })
  chat_history.extend([HumanMessage(content=question), ai_msg])

### Streaming

In [19]:
for chunk in rag_chain.stream({
  "question": "What is Task Decomposition", "chat_history": []
}):
  print(chunk.content, flush=True, end='')

Task decomposition involves breaking down a complex task into smaller and simpler steps to make it more manageable for an agent or model. Techniques like Chain of Thought (CoT) and Tree of Thoughts help in decomposing hard tasks into multiple manageable tasks by guiding the model to think step by step or explore multiple reasoning possibilities at each step. Task decomposition can be done using simple prompting, task-specific instructions, or human inputs to guide the agent or model in achieving the overall task goal.

### Per-User Retrieval

### Citations

### Use Agents

In [17]:
tools = [
  qdrant_instance.retriever_tool
]

agent_prompt = prompts.prompt_agent_openai_tools

agent = agents.create_openai_tools_agent(
  llm=chat_models.chat_openai,
  tools=tools,
  prompt=agent_prompt,
)
agent_executor = agents.AgentExecutor(agent=agent, tools=tools, verbose=True)

In [25]:
agent_executor.invoke({"input": "hi, i am Bob"})



> Entering new AgentExecutor chain...
Hello Bob! How can I assist you today?

> Finished chain.


{'input': 'hi, i am Bob', 'output': 'Hello Bob! How can I assist you today?'}

### Use Local Models

# [RAG From Scratch](https://youtube.com/playlist?list=PLfaIDFEXuae2LXbO1_PKyVJiQ23ZztA0x&si=dE6TOhGs5KMC1zc7)

[Git](https://github.com/langchain-ai/rag-from-scratch/tree/main)

## Basic Flow

- [Indexing](https://youtube.com/playlist?list=PLfaIDFEXuae2LXbO1_PKyVJiQ23ZztA0x&si=dE6TOhGs5KMC1zc7), [Slide](https://docs.google.com/presentation/d/1MhsCqZs7wTX6P19TFnA9qRSlxH3u-1-0gWkhBiDG9lQ/edit#slide=id.p)
- [Retrieval](https://youtu.be/LxNVgdIz9sU?si=rmu8kYV1BH_hwEvo), [Slide](https://docs.google.com/presentation/d/124I8jlBRCbb0LAUhdmDwbn4nREqxSxZU1RF_eTGXUGc/edit#slide=id.g267060cc54f_0_0)
- [Generation](https://youtu.be/Vw52xyyFsB8?si=pQqUluFZUrxTnwZP), [Slide](https://docs.google.com/presentation/d/1eRJwzbdSv71e9Ou9yeqziZrz1UagwX8B1kL4TbL5_Gc/edit#slide=id.g2b46f2cb556_0_0)

## Query Translation

### [Multi Query](https://youtu.be/JChPi0CRnDY?si=wEgjcc0NHINTvQVh), [Slide](https://docs.google.com/presentation/d/15pWydIszbQG3Ipur9COfTduutTZm6ULdkkyX-MNry8I/edit#slide=id.g268cd4ba153_0_0)



### [RAG Fusion](https://youtu.be/77qELPbNgxA?si=uyfzuemn02ktS2xe), [Slide](https://docs.google.com/presentation/d/1EwykmdVSQqlh6XpGt8APOMYp4q1CZqqeclAx61pUcjI/edit#slide=id.g268cfa48f45_0_0)



### [Decomposition](https://youtu.be/h0OPWlEOank?si=jU3DecxsmxDWi9az), [Slide](https://docs.google.com/presentation/d/1O97KYrsmYEmhpQ6nkvOVAqQYMJvIaZulGFGmz4cuuVE/edit#slide=id.g268fdc1fda2_0_0)



### [Step Back](https://youtu.be/xn1jEjRyJ2U?si=63WfDLTQwBKmUsdW), [Slide](https://docs.google.com/presentation/d/1L0MRGVDxYA1eLOR0L_6Ze1l2YV8AhN1QKUtmNA-fJlU/edit#slide=id.g268cfa65240_0_0)



### [HyDE](https://youtu.be/SaDzIVkYqyY?si=7tFx5bpTiBpy5KkV), [Slide](https://docs.google.com/presentation/d/10MmB_QEiS4m00xdyu-92muY-8jC3CdaMpMXbXjzQXsM/edit#slide=id.g2b872e9a17e_0_0)

### [Routing](https://youtu.be/pfpIndq7Fi8?si=m6SerpLuJdKzIV6A), [Slide](https://docs.google.com/presentation/d/1kC6jFj8C_1ZXDYcFaJ8vhJvCYEwxwsVqk2VVeKKuyx4/edit#slide=id.g26bc3116f45_0_0)



### [Query Structuring](https://youtu.be/kl6NwWYxvbM?si=Vm0MiQL13kI0nr-Q), [Blog](https://blog.langchain.dev/query-construction/)

## Indexing

### [Multi-Representation Indexing](https://youtu.be/gTCU9I6QqCE?si=jQ3Aj9ko3DYVQ1vU), [Slide](https://blog.langchain.dev/semi-structured-multi-modal-rag/)

### [RAPTOR](https://youtu.be/z_6EeA2LDSw?si=E09-N68W93TgBNBC), [Code](https://github.com/langchain-ai/langchain/blob/master/cookbook/RAPTOR.ipynb)

### [ColBERT](https://youtu.be/cN6S0Ehm7_8?si=LGBLo-VUonJMXnmR)

# end